# S3 - Generate Chi Value Expectation Surface

This notebook is part of the Supplementary Material provided for the paper
_Mapping indicators of cultural ecosystem services use in urban green spaces based on text classification of geosocial media data_ published in the Ecosystem Services: Science, Policy & Practice Journal. This includes the HTML conversions of a series of three Jupyter notebooks as follows: 

    1. S1_GSM_Data_Processing&LanguageModelTraining.html
    2. S2_GSM_Data_TextClassification.html
    3. S3_Generate_ChiValueExpectationSurface.html
https://doi.org/10.1016/j.ecoser.2022.101508

In this Notebook the following processes are addressed:

    1. Spatial Join between classified geosocial media posts located in green spaces and a hexagonal grid for the city of Dresden
    2. Aggregate classified geosocial media posts and calculate photo-user-days for each grid cell
    3. Compute chi expectation surface

**INPUT DATA:**
- hexagonal grid with the cell side length of 250 meters (.geojson file)  - was generated using ArcGIS Pro (version 2.9.3)
- classified Instagram and Flickr textual annotations in English and German (.csv file) that are located in the green spaces of Dresden
    
**OUTPUT:**
- Grid geometry with statistics - PUD, chi expectation values (.geojson file)


In [2]:
import datetime as dt
from IPython.display import clear_output, display, Markdown
date = dt.date.today()
display(Markdown(f'**Last update: {date}**'))

**Last update: 2023-01-02**

## 1. Preparations

### 1.1. Load dependencies

 - geopandas - version 0.10.2
 - numpy - version 1.21.3
 - pandas - version 1.3.3
 - scipy - version 1.7.1

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import sys
from pathlib import Path
import numpy as np

In [ ]:
INPUT = Path.cwd() / '01_Input'
OUTPUT = Path.cwd() / '02_Output'

### 1.2. Read the .geoJSON grid file into a GeoPandas geodataframe

In [ ]:
grid_file = INPUT/'HexagonGrid_250m.geojson'
grid = gpd.read_file(grid_file)
polygon_id = "GRID_ID"
print(f'{grid.crs}')

### 1.3. Parse the classified geosocial media posts into a pandas dataframe

We are interested in the analysis of the geosocial media posts published in the urban green spaces of Dresden.  To indentify them we performed a spatial join between a dataset containing all publicly accessible green spaces in the city of Dresden (Cakir et al., 2021) and the classified Flickr and Instagram dataset (output of the notebook  [S2_GSM_Data_TextClassification.html](./S2_GSM_Data_TextClassification.ipynb). This operation was conducted using ArcGIS Pro (version 2.9.3).

In [ ]:
gsm_classified_file = OUTPUT/'DD_FlickrInsta_Class_Intersect.csv'
cols = ['origin_id', 'latitude', 'longitude', 'post_guid', 'user_guid', 'post_date','tags','post_title','post_body','post_text','aesthetic','wildlife']
dtypes={'origin_id': str,'latitude': float, 'longitude': float,'post_guid':str, 'user_guid': str, 'post_date': str,'tags':str,'post_title':str,'post_body':str, 'post_text':str, 'aesthetic': float, 'wildlife':float}
df = pd.read_csv(gsm_classified_file,usecols=cols, dtype=dtypes, encoding = 'utf-8')
print(f'{len(df)} classified posts published within the green spaces of Dresden')

### 1.4. Transform pandas df to geopandas geodataframe and project it to the same coordinate system as the ugs geodataframe (UTM Zone 33N)

In [ ]:
crs_wgs = "EPSG:4326"
crs_proj = "EPSG:32633"

# transform pandas df into geopandas geodataframe
gsm_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = crs_wgs)

# project point gdf to UTM Zone 33N
gsm=gsm_gdf.to_crs(crs_proj)

# visualize to check if transformation worked
base = grid.plot(figsize=(20,25), color='white', edgecolor='grey', linewidth=0.2)
plot = gsm.plot(ax=base, color='blue', edgecolor ='black', linewidth=0.1)


### 1.5. Preparations for the calculation of photo-user-days (PUD) 

In [ ]:
def create_user_days(gsm_df):
    """ Generates a combination of day + month + user as a series"""

    #concatenate user_guid + day + month + year
    gsm_df["post_date"] = pd.to_datetime(gsm_df["post_date"])
    UD = gsm_df.apply(lambda x: str(x["post_date"].year) + str(x["post_date"].day) 
                   + str(x["post_date"].month) + "_" + str(x["user_guid"]), axis=1)

    return UD

# add userday column
gsm["userday"] = create_user_days(gsm)
gsm.head()

## 2. Perform Spatial Join & Calculate Statistics

Performs spatial join between the geosocial media posts and the hexagonal grid  and calculates number of posts (UP) and photo-user-days (PUD) for each single grid cell

In [ ]:
def count_UP_per_cell(grid_join, grid_stats, post_id_col = "post_guid", group_col = "GRID_ID"):
    up = pd.DataFrame(grid_join.groupby(group_col)[post_id_col].nunique())
    grid_stats = pd.merge(grid_stats, up, left_on= group_col, right_index=True)

    return grid_stats

def count_PUD_per_cell(grid_join, grid_stats, userday_col = "userday", group_col = "GRID_ID"):
    pud = pd.DataFrame(grid_join.groupby(group_col)[userday_col].nunique())
    grid_stats = pd.merge(grid_stats, pud, left_on= group_col, right_index=True)

    return grid_stats

#### 2.1. Calculate total UP & PUD

In [ ]:
#perform spatial join using geopandas
grid_join = gpd.sjoin(grid, gsm,  how="left", predicate="intersects")
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP","userday": "PUD"}, inplace=True)
grid.head()

#### 2.2. Calculate UP_Insta & PUD_Insta

In [ ]:
# Join Instagram posts to intersecting grid; left join which returns all combinations
gsm_insta = gsm[gsm.origin_id == "1"]
grid_join = gpd.sjoin(grid, gsm_insta,  how="left", predicate="intersects")
len(gsm_insta)

In [ ]:
#calculate UPs and PUDs
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Insta","userday": "PUD_Insta"}, inplace=True)
grid.head()

#### 2.3. Calculate UP_Flickr & PUD_Flickr

In [ ]:
# Join Flickr posts to intersecting grid; left join which returns all combinations
gsm_flickr = gsm[gsm.origin_id == "2"]
grid_join = gpd.sjoin(grid, gsm_flickr,  how="left", predicate="intersects")
len(gsm_flickr)

In [ ]:
#calculate UP_Flickr  and PUD_Flickr
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Flickr","userday": "PUD_Flickr"}, inplace=True)
grid.head()

#### 2.4. Calculate UP_Aesthetic & PUD_Aesthetic

In [ ]:
# Join all aesthetic classified posts to intersecting grid; left join which returns all combinations
gsm_aesthetic = gsm[gsm.aesthetic == 1]
grid_join = gpd.sjoin(grid, gsm_aesthetic,  how="left", predicate="intersects")
len(gsm_aesthetic)

In [ ]:
#calculate UP_Aesthetic and PUD_Aesthetic
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Aesthetic","userday": "PUD_Aesthetic"}, inplace=True)
grid.head()

#### 2.5. Calculate UP_Aesthetic_Insta & PUD_Aesthetic_Insta

In [ ]:
# Join Instagram aesthetic posts to intersecting grid; left join which returns all combinations
gsm_a_insta = gsm_aesthetic[gsm_aesthetic.origin_id == "1"]
grid_join = gpd.sjoin(grid, gsm_a_insta,  how="left", predicate="intersects")
len(gsm_a_insta)

In [ ]:
#calculate UP_Aesthetic_Insta and PUD_Aesthetic_Insta
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Aesthetic_Insta","userday": "PUD_Aesthetic_Insta"}, inplace=True)
grid.head()

#### 2.6. Calculate UP_Aesthetic_Flickr & PUD_Aesthetic_Flickr

In [ ]:
# Join Flickr aesthetic posts to intersecting grid; left join which returns all combinations
gsm_a_flickr = gsm_aesthetic[gsm_aesthetic.origin_id == "2"]
grid_join = gpd.sjoin(grid, gsm_a_flickr,  how="left", predicate="intersects")
len(gsm_a_flickr)

In [ ]:
#calculate UP_Aesthetic_Flickr and PUD_Aesthetic_Flickr
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Aesthetic_Flickr","userday": "PUD_Aesthetic_Flickr"}, inplace=True)
grid.head()

#### 2.7. Calculate UP_Wildlife & PUD_Wildlife

In [ ]:
# Join wildlife classified posts to intersecting grid; left join which returns all combinations
gsm_wildlife = gsm[gsm.wildlife == 1]
grid_join = gpd.sjoin(grid, gsm_a_flickr,  how="left", predicate="intersects")
len(gsm_wildlife)

In [ ]:
#calculate UP_Wildlife and PUD_Wildlife
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Wildlife","userday": "PUD_Wildlife"}, inplace=True)
grid.head()

#### 2.8. Calculate UP_Wildlife_Insta & PUD_Wildlife_Insta

In [ ]:
# Join Instagram wildlife posts to intersecting grid; left join which returns all combinations
gsm_w_insta = gsm_wildlife[gsm_wildlife.origin_id == '1']
grid_join = gpd.sjoin(grid, gsm_a_insta,  how="left", predicate="intersects")
len(gsm_w_insta)

In [ ]:
#calculate UP_Wildlife_Insta and PUD_Wildlife_Insta
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Wildlife_Insta","userday": "PUD_Wildlife_Insta"}, inplace=True)
grid.head()

#### 2.9. Calculate UP_Wildlife_Flickr & PUD_Wildlife_Flickr

In [ ]:
# Join Flickr wildlife posts to intersecting grid; left join which returns all combinations
gsm_w_flickr = gsm_wildlife[gsm_wildlife.origin_id == '2']
grid_join = gpd.sjoin(grid, gsm_a_flickr,  how="left", predicate="intersects")
len(gsm_w_flickr)

In [ ]:
#calculate UP_Wildlife_Insta and PUD_Wildlife_Insta
grid = count_UP_per_cell(grid_join, grid, post_id_col = "post_guid", group_col = polygon_id)
grid = count_PUD_per_cell(grid_join, grid, userday_col="userday", group_col=polygon_id)
grid.rename(columns={"post_guid":"UP_Wildlife_Flickr","userday": "PUD_Wildlife_Flickr"}, inplace=True)
grid.head()

Save grid geodatabase to GeoJSON file

In [ ]:
#save grid to geojson file
grid.to_file(OUTPUT/"Grid250m_Statistics_IntersectUGS.geojson", driver = "GeoJSON")

## 3. Calculate expectation (signed chi-value) statistics

To identify UGS which are underepresented/overrepresented for each of the two CES indicators we will calculate 
- for each category (aesthetic & wildlife) and each data source (Instagram & Flickr) we will calculate the expectation value


In [ ]:
#open geojson file
data_stat_file = OUTPUT/"Grid250m_Statistics_IntersectUGS.geojson"
gdf = gpd.read_file(data_stat_file)
gdf.head()

### 3.1.Calculate total UP & UD for whole dataset and for Insta and Flickr subsets

In [ ]:
N_UP= sum(gdf.UP)
N_PUD = sum(gdf.PUD)
#Instagram
NI_UP = sum(gdf.UP_Insta)
NI_PUD = sum (gdf.PUD_Insta)
#Flickr
NF_UP = sum(gdf.UP_Flickr)
NF_PUD = sum(gdf.PUD_Flickr)

print(N_UP,N_PUD, NI_UP,NI_PUD,NF_UP,NF_PUD)

### 3.2.Calculate total numbers for each category

In [ ]:
n_UP_A = sum(gdf.UP_Aesthetic)
n_PUD_A = sum(gdf.PUD_Aesthetic)
n_UP_W = sum(gdf.UP_Wildlife)
n_PUD_W = sum(gdf.PUD_Wildlife)

#Instagram subset
nI_UP_A = sum(gdf.UP_Aesthetic_Insta)
nI_PUD_A = sum(gdf.PUD_Aesthetic_Insta)
nI_UP_W = sum(gdf.UP_Wildlife_Insta)
nI_PUD_W = sum(gdf.PUD_Wildlife_Insta)

#Flickr subset
nF_UP_A = sum(gdf.UP_Aesthetic_Flickr)
nF_PUD_A = sum(gdf.PUD_Aesthetic_Flickr)
nF_UP_W = sum(gdf.UP_Wildlife_Flickr)
nF_PUD_W = sum(gdf.PUD_Wildlife_Flickr)

### Calculate signed chi value for each indicator (UP & PUD) for each of the activities (aesthetic & wildlife) 

In [ ]:
# taking into account the whole geosocial media data set
#aesthetic
gdf['A_UP_chi']= ((gdf.UP_Aesthetic * N_UP/n_UP_A) - gdf.UP)/np.sqrt(gdf.UP).astype('float64')
gdf['A_PUD_chi'] = ((gdf.PUD_Aesthetic * N_PUD/n_PUD_A) - gdf.PUD)/np.sqrt(gdf.PUD).astype('float64')
#wildlife
gdf['W_UP_chi']= ((gdf.UP_Wildlife * N_UP/n_UP_W) - gdf.UP)/np.sqrt(gdf.UP).astype('float64')
gdf['W_PUD_chi'] = ((gdf.PUD_Wildlife * N_PUD/n_PUD_W) - gdf.PUD)/np.sqrt(gdf.PUD).astype('float64')

In [ ]:
#calculate signed chi value taking into account just the Instagram subset
#aesthetic
gdf['AI_UP_chi'] = ((gdf.UP_Aesthetic_Insta * NI_UP/nI_UP_A) - gdf.UP_Insta)/np.sqrt(gdf.UP_Insta).astype('float64')
gdf['AI_PUD_chi'] = ((gdf.PUD_Aesthetic_Insta * NI_PUD/nI_PUD_A) - gdf.PUD_Insta)/np.sqrt(gdf.PUD_Insta).astype('float64')
#wildlife
gdf['WI_UP_chi'] = ((gdf.UP_Wildlife_Insta * NI_UP/nI_UP_W) - gdf.UP_Insta)/np.sqrt(gdf.UP_Insta).astype('float64')
gdf['WI_PUD_chi'] = ((gdf.PUD_Wildlife_Insta * NI_PUD/nI_PUD_W) - gdf.PUD_Insta)/np.sqrt(gdf.PUD_Insta).astype('float64')

In [ ]:
#calculate signed chi value taking into account just the Flickr subset
#aesthetic
gdf['AF_UP_chi'] = ((gdf.UP_Aesthetic_Flickr * NF_UP/nF_UP_A) - gdf.UP_Flickr)/np.sqrt(gdf.UP_Flickr).astype('float64')
gdf['AF_PUD_chi'] = ((gdf.PUD_Aesthetic_Flickr * NF_PUD/nF_PUD_A) - gdf.PUD_Flickr)/np.sqrt(gdf.PUD_Flickr).astype('float64')
#wildlife
gdf['WF_UP_chi'] = ((gdf.UP_Wildlife_Flickr * NF_UP/nF_UP_W) - gdf.UP_Flickr)/np.sqrt(gdf.UP_Flickr).astype('float64')
gdf['WF_PUD_chi'] = ((gdf.PUD_Wildlife_Flickr * NF_PUD/nF_PUD_W) - gdf.PUD_Flickr)/np.sqrt(gdf.PUD_Flickr).astype('float64')


In [ ]:
gdf.head()

In [ ]:
#save grid to geojson file
gdf.to_file(OUTPUT/"Grid250m_Chi.geojson", driver = "GeoJSON")

#### References

1. Wartmann, F.M., Mackaness, W.A., 2020. Describing and mapping where people experience tranquillity. An exploration based on interviews and Flickr photographs. Landscape Research 1–20. https://doi.org/10.1080/01426397.2020.1749250
2. Wood, J., Dykes, J., Slingsby, A., Clarke, K., 2007. Interactive Visual Exploration of a Large Spatio-temporal Dataset: Reflections on a Geovisualization Mashup. IEEE Trans. Visual. Comput. Graphics 13, 1176–1183. https://doi.org/10.1109/TVCG.2007.70570
3. Wood, S.A., Guerry, A.D., Silver, J.M., Lacayo, M., 2013. Using social media to quantify nature-based tourism and recreation. Sci Rep 3, 2976. https://doi.org/10.1038/srep02976

